In [ ]:
import pandas as pd
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
import plotly.express as px

import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
g1 = pd.read_csv("/content/drive/MyDrive/Working in August 2023/Plant_1_Generation_Data.csv")
g2 = pd.read_csv("/content/drive/MyDrive/Working in August 2023/Plant_2_Generation_Data.csv.zip")
w1 = pd.read_csv("/content/drive/MyDrive/Working in August 2023/Plant_1_Weather_Sensor_Data.csv")
w2 = pd.read_csv("/content/drive/MyDrive/Working in August 2023/Plant_2_Weather_Sensor_Data.csv")

# I prefer to work with lowercase column names
for df in [g1, g2, w1, w2]:
    df.columns = [col.lower() for col in df.columns]
    print(df.info(), "\n")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68778 entries, 0 to 68777
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date_time    68778 non-null  object 
 1   plant_id     68778 non-null  int64  
 2   source_key   68778 non-null  object 
 3   dc_power     68778 non-null  float64
 4   ac_power     68778 non-null  float64
 5   daily_yield  68778 non-null  float64
 6   total_yield  68778 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 3.7+ MB
None 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67698 entries, 0 to 67697
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date_time    67698 non-null  object 
 1   plant_id     67698 non-null  int64  
 2   source_key   67698 non-null  object 
 3   dc_power     67698 non-null  float64
 4   ac_power     67698 non-null  float64
 5   daily_yield  67698 non-null  float64
 

In [ ]:
g1

,date_time,plant_id,source_key,dc_power,ac_power,daily_yield,total_yield
0,2020-05-15 00:00:00,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.000,6259559.0
1,2020-05-15 00:00:00,4135001,1IF53ai7Xc0U56Y,0.0,0.0,0.000,6183645.0
2,2020-05-15 00:00:00,4135001,3PZuoBAID5Wc2HD,0.0,0.0,0.000,6987759.0
3,2020-05-15 00:00:00,4135001,7JYdWkrLSPkdwr4,0.0,0.0,0.000,7602960.0
4,2020-05-15 00:00:00,4135001,McdE0feGgRqW7Ca,0.0,0.0,0.000,7158964.0
...,...,...,...,...,...,...,...
68773,2020-06-17 23:45:00,4135001,uHbuxQJl8lW7ozc,0.0,0.0,5967.000,7287002.0
68774,2020-06-17 23:45:00,4135001,wCURE6d3bPkepu2,0.0,0.0,5147.625,7028601.0
68775,2020-06-17 23:45:00,4135001,z9Y9gH1T5YWrNuG,0.0,0.0,5819.000,7251204.0
68776,2020-06-17 23:45:00,4135001,zBIq5rxdHJRwDNY,0.0,0.0,5817.000,6583369.0


In [ ]:
g2

,date_time,plant_id,source_key,dc_power,ac_power,daily_yield,total_yield
0,2020-05-15 00:00:00,4136001,4UPUqMRk7TRMgml,0.0,0.0,9425.000000,2.429011e+06
1,2020-05-15 00:00:00,4136001,81aHJ1q11NBPMrL,0.0,0.0,0.000000,1.215279e+09
2,2020-05-15 00:00:00,4136001,9kRcWv60rDACzjR,0.0,0.0,3075.333333,2.247720e+09
3,2020-05-15 00:00:00,4136001,Et9kgGMDl729KT4,0.0,0.0,269.933333,1.704250e+06
4,2020-05-15 00:00:00,4136001,IQ2d7wF4YD8zU1Q,0.0,0.0,3177.000000,1.994153e+07
...,...,...,...,...,...,...,...
67693,2020-06-17 23:45:00,4136001,q49J1IKaHRwDQnt,0.0,0.0,4157.000000,5.207580e+05
67694,2020-06-17 23:45:00,4136001,rrq4fwE8jgrTyWY,0.0,0.0,3931.000000,1.211314e+08
67695,2020-06-17 23:45:00,4136001,vOuJvMaM2sgwLmb,0.0,0.0,4322.000000,2.427691e+06
67696,2020-06-17 23:45:00,4136001,xMbIugepa2P7lBB,0.0,0.0,4218.000000,1.068964e+08


In [ ]:
w1

,date_time,plant_id,source_key,ambient_temperature,module_temperature,irradiation
0,2020-05-15 00:00:00,4135001,HmiyD2TTLFNqkNe,25.184316,22.857507,0.0
1,2020-05-15 00:15:00,4135001,HmiyD2TTLFNqkNe,25.084589,22.761668,0.0
2,2020-05-15 00:30:00,4135001,HmiyD2TTLFNqkNe,24.935753,22.592306,0.0
3,2020-05-15 00:45:00,4135001,HmiyD2TTLFNqkNe,24.846130,22.360852,0.0
4,2020-05-15 01:00:00,4135001,HmiyD2TTLFNqkNe,24.621525,22.165423,0.0
...,...,...,...,...,...,...
3177,2020-06-17 22:45:00,4135001,HmiyD2TTLFNqkNe,22.150570,21.480377,0.0
3178,2020-06-17 23:00:00,4135001,HmiyD2TTLFNqkNe,22.129816,21.389024,0.0
3179,2020-06-17 23:15:00,4135001,HmiyD2TTLFNqkNe,22.008275,20.709211,0.0
3180,2020-06-17 23:30:00,4135001,HmiyD2TTLFNqkNe,21.969495,20.734963,0.0


In [ ]:
w2

,date_time,plant_id,source_key,ambient_temperature,module_temperature,irradiation
0,2020-05-15 00:00:00,4136001,iq8k7ZNt4Mwm3w0,27.004764,25.060789,0.0
1,2020-05-15 00:15:00,4136001,iq8k7ZNt4Mwm3w0,26.880811,24.421869,0.0
2,2020-05-15 00:30:00,4136001,iq8k7ZNt4Mwm3w0,26.682055,24.427290,0.0
3,2020-05-15 00:45:00,4136001,iq8k7ZNt4Mwm3w0,26.500589,24.420678,0.0
4,2020-05-15 01:00:00,4136001,iq8k7ZNt4Mwm3w0,26.596148,25.088210,0.0
...,...,...,...,...,...,...
3254,2020-06-17 22:45:00,4136001,iq8k7ZNt4Mwm3w0,23.511703,22.856201,0.0
3255,2020-06-17 23:00:00,4136001,iq8k7ZNt4Mwm3w0,23.482282,22.744190,0.0
3256,2020-06-17 23:15:00,4136001,iq8k7ZNt4Mwm3w0,23.354743,22.492245,0.0
3257,2020-06-17 23:30:00,4136001,iq8k7ZNt4Mwm3w0,23.291048,22.373909,0.0


In [ ]:
# convert date-times to pandas datetime format
g1['date_time']= pd.to_datetime(g1['date_time'],format='%d-%m-%Y %H:%M')
g2['date_time']= pd.to_datetime(g2['date_time'],format='%Y-%m-%d %H:%M:%S')
w1['date_time']= pd.to_datetime(w1['date_time'],format='%Y-%m-%d %H:%M:%S')
w2['date_time']= pd.to_datetime(w2['date_time'],format='%Y-%m-%d %H:%M:%S')

for df in [g1, g2, w1, w2]:
    print(df.date_time.head(), "\n")

0   2020-05-15
1   2020-05-15
2   2020-05-15
3   2020-05-15
4   2020-05-15
Name: date_time, dtype: datetime64[ns] 

0   2020-05-15
1   2020-05-15
2   2020-05-15
3   2020-05-15
4   2020-05-15
Name: date_time, dtype: datetime64[ns] 

0   2020-05-15 00:00:00
1   2020-05-15 00:15:00
2   2020-05-15 00:30:00
3   2020-05-15 00:45:00
4   2020-05-15 01:00:00
Name: date_time, dtype: datetime64[ns] 

0   2020-05-15 00:00:00
1   2020-05-15 00:15:00
2   2020-05-15 00:30:00
3   2020-05-15 00:45:00
4   2020-05-15 01:00:00
Name: date_time, dtype: datetime64[ns] 



In [ ]:
# check for null values
for df in [g1, g2, w1, w2]:
    print(df.isna().sum(), "\n")

date_time      0
plant_id       0
source_key     0
dc_power       0
ac_power       0
daily_yield    0
total_yield    0
dtype: int64 

date_time      0
plant_id       0
source_key     0
dc_power       0
ac_power       0
daily_yield    0
total_yield    0
dtype: int64 

date_time              0
plant_id               0
source_key             0
ambient_temperature    0
module_temperature     0
irradiation            0
dtype: int64 

date_time              0
plant_id               0
source_key             0
ambient_temperature    0
module_temperature     0
irradiation            0
dtype: int64 



In [ ]:
# there are no nulls, but are there date-times that are completely missing from some sensors?
Counter(g1.source_key)

Counter({'1BY6WEcLGh8j5v7': 3154,
         '1IF53ai7Xc0U56Y': 3119,
         '3PZuoBAID5Wc2HD': 3118,
         '7JYdWkrLSPkdwr4': 3133,
         'McdE0feGgRqW7Ca': 3124,
         'VHMLBKoKgIrUVDU': 3133,
         'WRmjgnKYAwPKWDb': 3118,
         'ZnxXDlPa8U1GXgE': 3130,
         'ZoEaEvLYb1n2sOq': 3123,
         'adLQvlD726eNBSB': 3119,
         'bvBOhCH3iADSZry': 3155,
         'iCRJl6heRkivqQ3': 3125,
         'ih0vzX44oOqAx2f': 3130,
         'pkci93gMrogZuBj': 3125,
         'rGa61gmuvPhdLxV': 3124,
         'sjndEbLyjtCKgGv': 3124,
         'uHbuxQJl8lW7ozc': 3125,
         'wCURE6d3bPkepu2': 3126,
         'z9Y9gH1T5YWrNuG': 3126,
         'zBIq5rxdHJRwDNY': 3119,
         'zVJPv84UY57bAof': 3124,
         'YxYtjZvoooNbGkE': 3104})